In [1]:
import os
import numpy as np
import tensorflow as tf
from vgg16 import *
from classes import class_names

In [2]:
# Misc
LOG_DIR = './log/'

# Training Parameters
LEARNING_RATE = 0.005
MOMENTUM = 0.9
NUM_EPOCHS = 1
NUM_STEPS = 500
BATCH_SIZE = 128
DISPLAY_STEP = 10

# Network Parameters
NUM_CLASSES = 1000
KEEP_RATE = 0.75

In [3]:
def read_image (path):
    # Read Image
    img_file = tf.read_file(path)
    img_decoded = tf.image.decode_jpeg(img_file, channels=3)
    img_resized = tf.image.resize_images(img_decoded, [224, 224])
    
    # Convert to BGR <-- Because the weights were trained from opencv images
    img = tf.reverse(img_resized, axis=[-1])
    
    # Subtract from mean
    imagenet_mean = np.array([104., 117., 124.])
    img -= imagenet_mean
    
    return img
    

In [4]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    data = tf.data.Dataset.from_tensor_slices(np.array(['../test_image/sealion.jpeg']))
    data = data.map(read_image).batch(1)
    iterator = data.make_one_shot_iterator()
    next_el = iterator.get_next()
    
    #images = tf.placeholder(tf.float32, [None, 227, 227, 3])
    #labels = tf.placeholder(tf.float32, [None, NUM_CLASSES])
    keep_prob = tf.placeholder(tf.float32)
    
    # Build Model
    vgg = MY_VGG16(x=next_el, keep_rate=keep_prob, 
                         num_classes=NUM_CLASSES, skip_layers=[],
                         weights_path='../vgg16.npy')
    vgg.build()
    
    # Logits and Predictions
    logits = vgg.logits
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    init = tf.global_variables_initializer()

In [5]:
with tf.Session(graph=model_graph) as sess:
    
    sess.run(init)
    
    vgg.load_weights(session=sess, encoding='latin1')
    
    pred = sess.run(prediction, feed_dict={keep_prob: 1.0})
    
    print(class_names[pred['classes'][0]])
    print(pred['prob'].max())

sea lion
0.98521477


In [5]:
writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'model_1'), graph=model_graph)